In [1]:
PROJECT_ID = "sunlit-analyst-309609"
%env GCLOUD_PROJECT=$PROJECT_ID
%load_ext google.cloud.bigquery

env: GCLOUD_PROJECT=sunlit-analyst-309609


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tempfile

import numpy as np
import pandas as pd
from multiprocessing import Pool
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.layers import Dense

from google.cloud import bigquery
LOCATION = "us"

In [3]:
from google.cloud import bigquery
from google.cloud import bigquery_storage

bqclient = bigquery.Client()
bqstorageclient = bigquery_storage.BigQueryReadClient()

## Testing 

This is a testing to see whether we can directly choose columns in pandas correctly so we don't have to call from biquery for each and every record, which is much more expensive in terms of time and cost than fetching all into pandas before doing manipulation. 

In [5]:
# %%bigquery df --use_bqstorage_api
# SELECT * EXCEPT (BB, CC, label),
# FROM `sunlit-analyst-309609.training_set.train_table_0` a

Downloading: 100%|██████████| 125600/125600 [00:10<00:00, 11711.64rows/s]


In [6]:
# a = np.array(df.loc[df["AAAA"] == 2])[:, 1:].flatten()
# a

array([0.5008592 , 0.7240773 , 0.80981546, ..., 1.00277082, 0.99809756,
       0.99494013])

In [28]:
# df.loc[df["AAAA"] == 1]

,AAAA,_0,_1,_2,_3,_4,_5,_6,_7,_8,...,_290,_291,_292,_293,_294,_295,_296,_297,_298,_299
84900,1,0.504461,0.724235,0.812151,0.874932,0.911667,0.941855,0.958407,0.970852,0.982835,...,1.000463,1.001674,0.998899,0.999176,0.998099,0.998742,0.999880,0.996169,0.998923,1.000007
84901,1,0.504430,0.721806,0.812968,0.873251,0.916751,0.939872,0.962460,0.971361,0.982808,...,0.998978,0.998557,0.999303,1.000704,0.999658,1.001793,0.998248,0.999198,1.001757,0.999277
84902,1,0.501610,0.723826,0.814340,0.877008,0.915292,0.942762,0.961914,0.976532,0.980890,...,1.000808,1.000796,1.000034,0.998788,1.003238,1.000333,0.996227,1.000776,1.000648,0.997978
84903,1,0.503904,0.724010,0.818815,0.875103,0.916524,0.941748,0.961877,0.979034,0.983156,...,1.000276,1.000465,1.003728,0.999946,1.001472,1.001827,1.000461,0.998925,1.004878,1.001769
84904,1,0.504843,0.722748,0.815991,0.874753,0.916074,0.941788,0.960547,0.974698,0.980729,...,0.999965,0.996774,0.997249,1.001838,1.000093,1.000956,0.999395,1.002111,1.002540,0.999237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84995,1,0.504291,0.719412,0.814134,0.875120,0.915518,0.943900,0.964959,0.976067,0.985015,...,1.002032,0.997889,1.004072,1.000762,0.998163,0.999086,1.001428,0.999111,1.000547,1.000504
84996,1,0.504484,0.721519,0.813124,0.876769,0.914985,0.943828,0.962988,0.976755,0.984346,...,0.998478,1.002097,1.005057,1.003266,0.998325,0.998699,1.001919,1.001269,0.999770,1.001834
84997,1,0.502171,0.719754,0.814797,0.877259,0.918352,0.944720,0.960349,0.975839,0.985077,...,0.995653,0.997679,1.001844,1.000364,0.997459,0.998965,0.996914,0.999438,1.000025,1.000820
84998,1,0.505423,0.719267,0.811880,0.872393,0.914324,0.941603,0.958838,0.974763,0.982734,...,1.000370,1.000844,0.998721,0.997314,1.001354,1.001642,1.001803,1.000853,1.001467,1.000607


In [29]:
# %%bigquery label --use_bqstorage_api
# SELECT DISTINCT AAAA, label
# FROM `sunlit-analyst-309609.training_set.train_table_0`

Downloading: 100%|██████████| 1256/1256 [00:01<00:00, 1135.60rows/s]


In [36]:
# b = np.array(label.loc[label["AAAA"] == 1])[:, 1:].squeeze()
# b

array(0.05427012)

In [69]:
# np.append(a, b).reshape((1, -1)).shape

(1, 30001)

## Confirmation success
Corfirmation is now success. Next is to create the csv file from which we could load from this. 

In [4]:
folder = "./csv_files/"

First, select all distinct AAAA that we would like to use as iterators. 

In [5]:
%%bigquery AAAAs --use_bqstorage_api
SELECT DISTINCT AAAA
FROM`sunlit-analyst-309609.training_set.train_table_0`
ORDER BY AAAA

Downloading: 100%|██████████| 1256/1256 [00:01<00:00, 994.40rows/s]


In [9]:
# for i in AAAAs.values.flatten():
#     print(i)

#     if i > 10:
#         break

1
2
3
4
5
6
7
8
9
10
11


In [4]:
from numba import jit

In [12]:
def flatten_to_csv(f) :
    for table_name in f:
        query_string = f"""
        SELECT * EXCEPT (BB, CC, label),
        FROM `sunlit-analyst-309609.training_set.train_table_{table_name}`
        """

        df = (
            bqclient.query(query_string)
            .result()
            .to_dataframe(bqstorage_client=bqstorageclient)
        )

        query_string = f"""
        SELECT DISTINCT AAAA, label
        FROM `sunlit-analyst-309609.training_set.train_table_{table_name}`
        """

        target = (
            bqclient.query(query_string)
            .result()
            .to_dataframe(bqstorage_client=bqstorageclient)
        )

        for AAAA in tqdm(AAAAs.values.flatten()):
            main_array = np.array(df.loc[df["AAAA"] == AAAA])[:, 1:].flatten()

            side_array = np.array(target.loc[target["AAAA"] == AAAA])[:, 1:].squeeze()

            total_array = np.append(main_array, side_array).reshape((1, -1))

            assert total_array.shape == (1, 30001)

            # Convert to pandas DataFrame
            total_array = pd.DataFrame(total_array, dtype="float")

            total_array.to_csv(folder + f"flatten_train_{table_name}.csv", header=False, index=False, mode="a", sep=",")

        print(f"\nDone with table {table_name}.\n")

    return None


# def call_multiprocessing(processes):
#     params = np.array_split(np.arange(55), processes)

#     with Pool(processes) as pool:
#         results = pool.map(flatten_to_csv, params)


# call_multiprocessing(2)
flatten_to_csv([3])

100%|██████████| 1256/1256 [02:37<00:00,  7.97it/s]


Done with table 3.



Let's assert the files and see that the shape is as we expected. Be aware that this is not an efficient test as you consider how much GB of data we are reading just to make the assertions. But since we are only running it for one time and never again we don't care that much, time is sufficient to spare (ahem, waste). 

In [11]:
np.array(pd.read_csv(f"./csv_files/flatten_train_3.csv", header=None)).shape

(187, 30001)

In [13]:
def test_ours(a):
    for table_name in tqdm(a):
        assert np.array(pd.read_csv(f"./csv_files/flatten_train_{table_name}.csv", header=None)).shape == (1256, 30001)

    return None

def driver_func():
    # processes = os.cpu_count()
    processes = 2
    params = np.array_split(np.arange(55), processes)

    with Pool(processes) as pool:
        results = pool.map(test_ours, params)


driver_func()

100%|██████████| 28/28 [13:41<00:00, 29.33s/it]


Next, we would like to upload them to bigquery dataset. Refer to https://www.kaggle.com/rtatman/uploading-csv-to-bigquery for the original code. 

In [85]:
# dataset_id = "flatten_train"
# bqclient.create_dataset(dataset_id)

In [102]:
# client = bqclient  # due to copying, uneven in naming. (inconsistency)

# def upload_to_bigquery(f):
#     for table_name in f:

#         table_id = f"flatten_train_{table_name}"
#         filename = f"./csv_files/flatten_train_{table_name}.csv"

#         # Create new table in that dataset
#         client.create_table(f"{PROJECT_ID}.{dataset_id}.{table_id}")
        
#         # Tell the client everything it needs to know to upload our csv. 
#         dataset_ref = client.dataset(dataset_id)
#         table_ref = dataset_ref.table(table_id)
#         job_config = bigquery.LoadJobConfig()
#         job_config.source_format = bigquery.SourceFormat.CSV
#         job_config.autodetect = True  # Actually no use since we are not saving a header. 

#         # Load csv into bigquery. 
#         with open(filename, "rb") as source_file:
#             job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

#         job.result()  # Waits for table load to complete. 

#         # Check everything works. 
#         print(f"Loaded {job.output_rows} rows into {dataset_id}:{table_id}.")
#         print(f"Finish loading table {table_name}.\n")


# def multiprocess_call(processes):
#     # processes = 2
#     params = np.array_split(np.arange(55), processes)

#     with Pool(processes) as pool:
#         pool.map(upload_to_bigquery, params)

In [103]:
# multiprocess_call(2)

BadRequest: 400 Number of columns 30001 is too many for table 'flatten_train_19_6cf8f528_a70c_4a62_bb6c_c787b543d081_source'. A table must have no more than 10000 columns.

Now you can go to bigquery and check that everything is there. Violà! 

Note that however the above process is not fail-safe. There had been ignored try-except blocks that can make it fail-safe but not in place. 

## Explanation

Well, it seems like it failed to load to BigQuery, so we have to load from csv instead. 

# Do the same for test files
Remember we have a different AAAAs for this so we need to load them. 

In [5]:
%%bigquery AAAAs --use_bqstorage_api
SELECT DISTINCT AAAA
FROM`sunlit-analyst-309609.test_set.test_table_0`
ORDER BY AAAA

Downloading: 100%|██████████| 539/539 [00:01<00:00, 353.61rows/s]


In [8]:
test_folder = "./csv_files_test/"


def flatten_to_csv(f) :
    for table_name in f:
        query_string = f"""
        SELECT * EXCEPT (BB, CC),
        FROM `sunlit-analyst-309609.test_set.test_table_{table_name}`
        """

        df = (
            bqclient.query(query_string)
            .result()
            .to_dataframe(bqstorage_client=bqstorageclient)
        )

        for AAAA in tqdm(AAAAs.values.flatten()):
            total_array = np.array(df.loc[df["AAAA"] == AAAA])[:, 1:].flatten().reshape((1, -1))

            assert total_array.shape == (1, 30000)

            # Convert to pandas DataFrame
            total_array = pd.DataFrame(total_array, dtype="float")

            total_array.to_csv(test_folder + f"flatten_test_{table_name}.csv", header=False, index=False, mode="a", sep=",")

        print(f"\nDone with table {table_name}.\n")

    return None


# def call_multiprocessing(processes):
#     params = np.array_split(np.arange(55), processes)

#     with Pool(processes) as pool:
#         results = pool.map(flatten_to_csv, params)


# call_multiprocessing(2)
flatten_to_csv(range(5, 55))

100%|██████████| 539/539 [01:06<00:00,  8.07it/s]



Done with table 5.



100%|██████████| 539/539 [01:06<00:00,  8.16it/s]



Done with table 6.



100%|██████████| 539/539 [01:06<00:00,  8.16it/s]


...


Done with table 52.



100%|██████████| 539/539 [00:56<00:00,  9.48it/s]



Done with table 53.



100%|██████████| 539/539 [00:56<00:00,  9.55it/s]


Done with table 54.



In [9]:
def test_ours(a):
    for table_name in tqdm(a):
        assert np.array(pd.read_csv(f"./csv_files_test/flatten_test_{table_name}.csv", header=None)).shape == (539, 30000)

    return None

def driver_func():
    # processes = os.cpu_count()
    processes = 2
    params = np.array_split(np.arange(55), processes)

    with Pool(processes) as pool:
        results = pool.map(test_ours, params)


# driver_func()